In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
from nltk import pos_tag
import string
import csv
import re

import json
from math import log
import sys
import nltk, re, pprint
from nltk import bigrams
import urllib
import copy
from nltk.tokenize import word_tokenize
import nltk.data
from math import log
import itertools

from bs4 import BeautifulSoup
import urllib.request

stopword_list = set(stopwords.words('english'))

In [14]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mohsen\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

DATA ADDRESS

In [99]:
add_news = 'Downloads/kerman/LDC2003T05'

In [72]:
def read_news(add):
    
    with open(add, 'r') as f:
        data = f.readlines()
    news=[]    
    n = len(data)
    #n = 1500
    for i in range(n):
        if data[i].find('<TEXT>')!= -1 :
            a_news = []
            while(i+1<n and data[i+1].find('<\TEXT>')==-1):
                while(i+1<n and data[i+1].find('<\P>')==-1):
                    if data[i+1].find('<P>')!=-1:
                        #the_str = 
                        a_news.append(data[i+2])
                    i = i+1
                    if i>=n:
                        break
            if  a_news != []:
                news.append(a_news)
            
    return news

In [76]:
data_news = read_news(add_news)

In [75]:
def expand_contractions(text, contraction_mapping):
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),
                                      flags=re.IGNORECASE | re.DOTALL)

def expand_match(contraction):
    match = contraction.group(0)
    first_char = match[0]
    expanded_contraction = contraction_mapping.get(match) \
    if contraction_mapping.get(match) \
    else contraction_mapping.get(match.lower())
    expanded_contraction = first_char + expanded_contraction[1:]
    
    return expanded_contraction

    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

def tokenize_text(text):
    tokens = word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    return tokens

def remove_special_characters(text):
    tokens = tokenize_text(text)
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    filtered_tokens = filter(None, [pattern.sub(' ', token) for token in tokens])
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

def remove_stopwords(text):
    tokens = tokenize_text(text)
    filtered_tokens = [token for token in tokens if token not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text


In [16]:
def pre_process(mytext):
    mytext = mytext.lower()
    mytext = remove_special_characters(mytext)
    mytext = remove_stopwords(mytext)
    return mytext

In [22]:
def score_sent(the_txt):

    the_txt = pre_process(the_txt)
    desired_google_queries = the_txt.split()

    the_score = 0
    for query in desired_google_queries:

        url = 'http://google.com/search?q=' + query

        req = urllib.request.Request(url, headers={'User-Agent' : "Magic Browser"})
        response = urllib.request.urlopen( req )
        html = response.read()

        soup = BeautifulSoup(html, 'html.parser')

        resultStats = soup.find(id="resultStats").string

        the_score = the_score + (int(((resultStats.replace('About ','').replace('results','')).replace(',',''))))
        n = len(desired_google_queries) 
        if n==0:
            finalScore =  0
        else :
            finalScore =  the_score/n
    return (finalScore)

In [94]:
import numpy as np
def summ_new_item(item_score, news, num_sent_sum):
    n = len(item_score)
    indexs = np.argsort(item_score)[n-num_sent_sum, n]
    my_sum = np.array(news[indexs])
    
    return my_sum

In [ ]:
all_sum = []
num_sent_sum=2

for jtem in data_news:
    item_score = []
    for item in jtem:
        item_score.append(score_sent(item))
    all_sum.append(summ_new_item(item_score, news, num_sent_sum))